In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import os
import cv2
import time
import tensorflow as tf

from detector import Detector

In [ ]:
MODEL_PATH = 'model.pb'
detector = Detector(MODEL_PATH, gpu_memory_fraction=0.5, visible_device_list='0')

# Get an image

In [ ]:
path = '/home/dan/datasets/COCO/images/val2017/'
i = os.listdir(path)[14]
path = path + i
image_array = cv2.imread(path)
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image_array)
image

# Show detections

In [ ]:
def draw_on_image(image, heatmaps, segmentation_masks):

    image_copy = image.copy()
    image_copy.putalpha(255)
    #draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image_copy.size
    
    m = segmentation_masks.min()
    M = segmentation_masks.max()
    segmentation_masks = (segmentation_masks - m)/(M - m)
    segmentation_masks = Image.fromarray((segmentation_masks*255.0).astype('uint8'))
    segmentation_masks= segmentation_masks.convert('RGB')
    segmentation_masks.putalpha(150)
    segmentation_masks = segmentation_masks.resize((width, height))
    segmentation_masks

    image_copy.alpha_composite(segmentation_masks)
    return image_copy

In [ ]:
image_array = cv2.resize(image_array, (640, 640))
heatmaps, segmentation_masks = detector(image_array, score_threshold=0.6)

In [ ]:
m = segmentation_masks.min()
M = segmentation_masks.max()
segmentation_masks = (segmentation_masks - m)/(M - m)
segmentation_masks = Image.fromarray((segmentation_masks*255.0).astype('uint8'))
segmentation_masks

# Measure speed

In [ ]:
times = []
for _ in range(110):
    start = time.perf_counter()
    boxes, labels, scores = detector(image_array, score_threshold=0.25)
    times.append(time.perf_counter() - start)
    
times = np.array(times)
times = times[10:]
print(times.mean(), times.std())